In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import sys
sys.path.append('../')
from mrcnn.utils import trim_zeros_graph
import numpy as np
import tensorflow as tf

np.set_printoptions(linewidth=100, precision=4)
sess = tf.InteractiveSession()

batch_size = 3
num_rois = 32
num_classes = 4
num_detections = 100
num_rois_per_image = 32
rois_per_image = 32

D:\Program Files\Anaconda3\envs\TF_gpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


###  Stack non-zero bbox infromaton from pred_array into stacked_array

In [31]:
# ps = pred_tensor[~tf.all(pred_tensor[:,2:6] == 0, axis=1)]
# print(' pred tensor shape is : ', pred_tensor.shape)

pt2 = pred_tensor[:,:,:,1:]

print(' pt2 shape is : ', pt2.shape)
# pt2_exp = tf.expand_dims(pt2,axis = 1)
# print(' pt2_exp shape is : ', pt2_exp.shape)
# print(pt2[0,0].eval())
# print(pt2[0,1].eval())
# print(pt2[0,2].eval())

pt2_reshape = tf.reshape( pred_tensor[:,:,:,1:] , [batch_size, num_classes * rois_per_image ,6])
print(' pt2_reshape shape is : ', pt2_reshape.get_shape())
 
# print(pt2_reshape[0].eval())
# print(pt2_reshape[1].eval())
# print(pt2_reshape[2].eval())

pt2_sum = tf.reduce_sum(tf.abs(pt2), axis=-1)
print(' pt2_sum shape ',pt2_sum.shape)
# print(pt2_sum[0].eval())

pt2_mask = tf.greater(pt2_sum , 0)
# print(' pt2_mask shape ', pt2_mask.get_shape())
# print(pt2_mask.eval())

pt2_ind  = tf.where(pt2_mask)
print(' pt2_ind shape  ', pt2_ind.get_shape())
# print(pt2_ind.eval())
# pt2_ind_float  =  tf.to_float(pt2_ind[:,0:1])

dense1 = tf.gather_nd( pt2, pt2_ind)
print(' dense1 shape ',dense1.get_shape())
# print(dense1.eval())

dense1 = tf.concat([tf.to_float(pt2_ind[:,0:1]), dense1],axis=1)
# print(' dense1 shape ',dense1.get_shape())
# print(dense1.eval())

# print(dense[1].eval())
# print(dense[2].eval())
# print(dense[3].eval())
stacked_list = tf.dynamic_partition(dense1, tf.to_int32(pt2_ind[:,0]),num_partitions = batch_size )
# print(len(dyn_part))      

for img in range(len(stacked_list) ):
    rois_in_image, cols  = tf.shape(stacked_list[img]).eval()
    print('\n ===> list item #', img, ' stacked_list[img] shape: ',rois_in_image, cols)   
#     print(stacked_list[img].eval())            
    
    stacked_list[img] = tf.pad(stacked_list[img],[[0,32-rois_in_image],[0,0]])
        
    print('tensor_list item pos padding :', tf.shape(stacked_list[img]).eval())
#     print(stacked_list[img].eval())

stacked_tensor = tf.stack(stacked_list)
print(tf.shape(stacked_tensor).eval())        
    

# print(stacked_list[img].eval())                
# experiment with scatter.....    
# pred_scatt = tf.scatter_nd(scatter_classes, pt2, [batch_size, num_classes, num_rois,7])    
# print('pred_scatt', pred_scatt.get_sh)

 pt2 shape is :  (3, 4, 32, 6)
 pt2_reshape shape is :  (3, 128, 6)
 pt2_sum shape  (3, 4, 32)
 pt2_ind shape   (?, 3)
 dense1 shape  (?, 6)

 ===> list item # 0  stacked_list[img] shape:  22 7
tensor_list item pos padding : [32  7]

 ===> list item # 1  stacked_list[img] shape:  28 7
tensor_list item pos padding : [32  7]

 ===> list item # 2  stacked_list[img] shape:  30 7
tensor_list item pos padding : [32  7]
[ 3 32  7]


###  Stack non-zero bbox infromaton from pred_array into stacked_array
### Method 2 -- slower 

In [6]:
# ps = pred_tensor[~tf.all(pred_tensor[:,2:6] == 0, axis=1)]
print(' pred tensor shape is : ', pred_tensor[:,:,:,1:].shape)

pt2_reshape = tf.reshape( pred_tensor[:,:,:,1:] , [batch_size, num_classes * rois_per_image ,6])

print(' pt2_reshape shape is : ', pt2_reshape.get_shape())
# print(pt2_reshape[0].eval())
# print(pt2_reshape[1].eval())
# print(pt2_reshape[2].eval())

pt2_sum = tf.reduce_sum(tf.abs(pt2_reshape), axis=-1)
print(' pt2_sum shape ',pt2_sum.shape)
# print(pt2_sum[0].eval())

pt2_mask = tf.greater(pt2_sum , 0)
print(' pt2_mask shape ', pt2_mask.get_shape())
# print(pt2_mask.eval())

pt2_ind  = tf.where(pt2_mask)
print(' pt2_ind shape  ', pt2_ind.get_shape())
# print(pt2_ind.eval())

dense1 = tf.gather_nd( pt2_reshape, pt2_ind)
# print(' dense1 shape ',dense1.get_shape())
# print(dense1.eval())

dense1 = tf.concat([tf.to_float(pt2_ind[:,0:1]), dense1],axis=1)
# print(' dense1 shape ',dense1.get_shape())
# print(dense1.eval())

# print(dense[1].eval())
# print(dense[2].eval())
# print(dense[3].eval())

stacked_list = tf.dynamic_partition(dense1, tf.to_int32(pt2_ind[:,0]),num_partitions = batch_size )
# print(len(dyn_part))

for img in range(len(stacked_list) ):
    rois_in_image, cols  = tf.shape(stacked_list[img]).eval()
    print('\n===> list item #', img, ' stacked_list[img] shape: ',rois_in_image, cols)   
#     print(stacked_list[img].eval())            
    
    stacked_list[img] = tf.pad(stacked_list[img],[[0,32-rois_in_image],[0,0]])
        
    print('tensor_list item pos padding :', tf.shape(stacked_list[img]).eval())
#     print(stacked_list[img].eval())

stacked_tensor = tf.stack(stacked_list)
print(tf.shape(stacked_tensor).eval())

 pred tensor shape is :  (3, 4, 32, 6)
 pt2_reshape shape is :  (3, 128, 6)
 pt2_sum shape  (3, 128)
 pt2_mask shape  (3, 128)
 pt2_ind shape   (?, 2)

===> list item # 0  stacked_list[img] shape:  22 7
tensor_list item pos padding : [32  7]

===> list item # 1  stacked_list[img] shape:  28 7
tensor_list item pos padding : [32  7]

===> list item # 2  stacked_list[img] shape:  30 7
tensor_list item pos padding : [32  7]
[ 3 32  7]


### Build coordinate meshgrid  pos_grid

Slightly faster method  - build meshgrid in the following dimensions to allow for tensorflow Multivariate Normal Dist
    
    width x height x batch_size x rois per image x 2

In [32]:
rois_per_image  = 32
img_h  = 128
img_w  = 128
X = tf.range(img_w, dtype=tf.int32)
Y = tf.range(img_h, dtype=tf.int32)
X, Y = tf.meshgrid(X, Y)
print( X.get_shape(), Y.get_shape())
# print( ' X : \n',X.eval())
# print( ' Y : \n',Y.eval())

# ## hear we repeat X and Y  batch_size x rois_per_image times
ones = tf.ones([batch_size, rois_per_image,1, 1], dtype = tf.int32)
print(' ones: ',ones.shape)                
# # ones = tf.expand_dims(ones,-1)
# print(' ones with exp dims ',ones.shape)

rep_X = ones * X
rep_Y = ones * Y 
# print(' ones_exp * X', ones.shape, '*', X.shape, '= ',rep_X.shape)
# print(' ones_exp * Y', ones.shape, '*', Y.shape, '= ',rep_Y.shape)

# # stack the X and Y grids 
bef_pos = tf.to_float(tf.stack([rep_X,rep_Y], axis = -1))
# print(' before transpse ', bef_pos.get_shape())
pos_grid_1 = tf.transpose(bef_pos,[2,3,0,1,4])
print(' after transpose ', pos_grid_1.get_shape())    


(128, 128) (128, 128)
 ones:  (3, 32, 1, 1)
 after transpose  (128, 128, 3, 32, 2)


In [33]:
# ## Check equality of the two methods 

# eq = tf.equal(pos_grid_1, pos_grid_2)
# print( ' grid and pos equal --> ', tf.reduce_all(eq).eval())


### Calculate mean and covariance matrices for gaussian distributions

In [34]:
# for ps in all_boxes:

ps = stacked_tensor
# for img, ps_init in enumerate(stacked_tensor):
# print('\n===> list memeber #', img, ' shape: ', ps_init.get_shape(), '  ',tf.shape(ps_init).eval())   
# rois_in_image, cols  = tf.shape(ps_init).eval()
# print(' ps \t rows :', rois_in_image, '\t cols: ', cols)

# ps = tf.pad(ps_init,[[0,32-rois_in_image],[0,0]])

# print('ps.shape is ', tf.shape(ps_init).eval())
# print(ps_init.eval())    
print('ps.shape is ', tf.shape(ps).eval())
# print(ps.eval())

width  = ps[:,:,5] - ps[:,:,3]
height = ps[:,:,4] - ps[:,:,2]
cx     = ps[:,:,3] + ( width  / 2.0)
cy     = ps[:,:,2] + ( height / 2.0)
means  = tf.stack((cx,cy),axis = -1)
covar  = tf.stack((width * 0.5 , height * 0.5), axis = -1)
# print(means.eval())
# print(covar.eval())

# print('width shape ',width.get_shape()) 
mns = means
cov = covar
# print(mns.eval())
tfd = tf.contrib.distributions
mvn = tfd.MultivariateNormalDiag(
    loc  = mns,
    scale_diag = cov)

print(' means shape ',means.get_shape(), '  ', tf.shape(means).eval())
print(' covar shape ',covar.get_shape(), '  ', tf.shape(covar).eval())
print(' from MVN :  \t mns shape      :', tf.shape(mns).eval(), ' \t cov shape : ', tf.shape(cov).eval())
print(' from MVN :  \t mean shape     :', tf.shape(mvn.mean()).eval(), '\t stddev shape', tf.shape(mvn.stddev()).eval())
print(' from MVN :  \t mvn.batch_shape:', mvn.batch_shape , '\t mvn.event_shape ',  mvn.event_shape)
# print(mvn.loc.eval())
print(' Linear OP shape', mvn.scale.shape, ' Linear Op batch shape ',mvn.scale.batch_shape)
print(' Linear op Range Dim  ', mvn.scale.range_dimension)
print(' Linear op Domain Dim ', mvn.scale.domain_dimension) 
    
inp = pos_grid_1
print(' >> input to MVN.PROB: pos_grid (meshgrid) shape: ', inp.get_shape())
# print(inp.eval())
# one_layer = tf.to_float(pos[0])
# print(one_layer[0].get_shape)

prob_grid = mvn.prob(inp)
print(' << output probabilities shape:' , prob_grid.get_shape())
# print(prob.eval())

#     eq = tf.equal(grid, pos)
#     print( ' pos and grid probabalitiy matricies equal  -->', tf.reduce_all(eq).eval())
trans_grid = tf.transpose(prob_grid,[2,3,0,1])


# gauss_tensor.append(trans_grid)
print(' trans_grid shape: ', trans_grid.shape)

gauss_grid = tf.where(tf.is_nan(trans_grid),  tf.zeros_like(trans_grid), trans_grid)

ps.shape is  [ 3 32  7]
 means shape  (3, ?, 2)    [ 3 32  2]
 covar shape  (3, ?, 2)    [ 3 32  2]
 from MVN :  	 mns shape      : [ 3 32  2]  	 cov shape :  [ 3 32  2]
 from MVN :  	 mean shape     : [ 3 32  2] 	 stddev shape [ 3 32  2]
 from MVN :  	 mvn.batch_shape: (?, ?) 	 mvn.event_shape  (2,)
 Linear OP shape (3, ?, 2, 2)  Linear Op batch shape  (3, ?)
 Linear op Range Dim   2
 Linear op Domain Dim  2
 >> input to MVN.PROB: pos_grid (meshgrid) shape:  (128, 128, 3, 32, 2)
 << output probabilities shape: (128, 128, 3, 32)
 trans_grid shape:  (3, 32, 128, 128)


### Build indicies to scatter probabiltiy distributions by class

In [35]:
class_inds = tf.to_int32(stacked_tensor[:,:,6])
batch_grid, roi_grid = tf.meshgrid( tf.range(batch_size, dtype=tf.int32), tf.range(num_rois, dtype=tf.int32), indexing = 'ij' )

print('class          shape: ', class_inds.shape)
print(class_inds.eval())
print('roi_grid shape',tf.shape(roi_grid).eval())
print(roi_grid.eval())
print('batch_grid     shape',tf.shape(batch_grid).eval())
print(batch_grid.eval())

scatter_classes = tf.stack([batch_grid, class_inds, roi_grid ],axis = -1)
print(scatter_classes.shape)
print(scatter_classes.eval())

gauss_scatt = tf.scatter_nd(scatter_classes, gauss_grid, [batch_size, num_classes, num_rois, 128, 128])
print(' gaussian_grid      :', gauss_grid.shape)
print(' gaussian scattered :', gauss_scatt.shape)

gauss_sum = tf.reduce_sum(gauss_scatt, axis=2)
print(' gaussian sum       :', gauss_sum.shape)

class          shape:  (3, ?)
[[0 0 0 0 0 0 0 1 1 1 1 1 1 2 2 2 2 2 2 3 3 3 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 3 3 3 3 3 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 3 3 3 3 3 0 0]]
roi_grid shape [ 3 32]
[[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31]]
batch_grid     shape [ 3 32]
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]]
(3, 32, 3)
[[[ 0  0  0]
  [ 0  0  1]
  [ 0  0  2]
  [ 0  0  3]
  [ 0  0  4]
  [ 0  0  5]
  [ 0  0  6]
  [ 0  1  7]
  [ 0  1  8]
  [ 0  1  9]
  [ 0  1 10]
  [ 0  1 11]
  [ 0  1 12]
  [ 0  2 13]
  [ 0  2 14]
  [ 0  2 1

In [36]:
gauss_scatt[0,0,0].eval()
srt = 10
end = 17
img = 0
item = 1
clss = tf.to_int32(pred_classes[img, item]).eval()
print(clss)
eq = tf.equal(gauss_scatt[img,clss,item],gauss_grid[img,item])

print( 'equal ', tf.reduce_all(eq).eval())

print(' gaussian_grid      :', gauss_grid.shape)
print(gauss_grid[img,item].eval())
print()

print(' gaussian scattered :', gauss_scatt.shape)
print(gauss_scatt[img,clss,item].eval())
print()

print(' gaussian sum       :', gauss_sum.shape)
print(gauss_sum[img,item].eval())

0
equal  True
 gaussian_grid      : (3, 32, 128, 128)
[[3.2667e-05 4.8889e-05 6.9581e-05 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 [3.4049e-05 5.0958e-05 7.2525e-05 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 [3.5469e-05 5.3083e-05 7.5549e-05 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 ...
 [5.6818e-05 8.5034e-05 1.2102e-04 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 [5.4964e-05 8.2258e-05 1.1707e-04 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 [5.3138e-05 7.9526e-05 1.1318e-04 ... 0.0000e+00 0.0000e+00 0.0000e+00]]

 gaussian scattered : (3, 4, 32, 128, 128)
[[3.2667e-05 4.8889e-05 6.9581e-05 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 [3.4049e-05 5.0958e-05 7.2525e-05 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 [3.5469e-05 5.3083e-05 7.5549e-05 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 ...
 [5.6818e-05 8.5034e-05 1.2102e-04 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 [5.4964e-05 8.2258e-05 1.1707e-04 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 [5.3138e-05 7.9526e-05 1.1318e-04 ... 0.0000e+00 0.0000e+00 0.0000e+00]]

 gaussian sum

In [460]:
# for img in range(3):
#     for idx in range(32):
#         compare = tf.equal(trans_grid[img,idx], trans_pos[img,idx])
#         print('compare shape' , compare.get_shape())
#         eq = tf.reduce_all(compare).eval()
#         print( img , '/' ,idx, ' TRANSPOSED pos and grid probabalitiy matricies equal  -->', eq)
#         if not eq :
#             print(trans_grid[img,idx].eval())
#             print(trans_pos[img,idx].eval())

# comapre = tf.equal(trans_grid[3], trans_pos[3])
# eq = tf.reduce_all(compare).eval()            
# print(eq)

In [31]:
for img in range(batch_size):
    for idx in range(rois_per_image):
        clss = tf.to_int32(pred_classes[img, idx]).eval()
        
        compare  = tf.equal(gauss_scatt[img,clss,idx],gauss_grid[img,idx])
        eq =  tf.reduce_all(eq)
        print('img:', img,  'idx' , idx, 'class ', clss , 'equal ', eq.eval())
        if not eq.eval():
            print(gauss_scatt[0,1,idx].eval())
            print(gauss_grid[0,idx].eval())

img: 0 idx 0 class  0 equal  True
img: 0 idx 1 class  0 equal  True
img: 0 idx 2 class  3 equal  True
img: 0 idx 3 class  0 equal  True
img: 0 idx 4 class  1 equal  True
img: 0 idx 5 class  1 equal  True
img: 0 idx 6 class  2 equal  True
img: 0 idx 7 class  2 equal  True
img: 0 idx 8 class  1 equal  True
img: 0 idx 9 class  0 equal  True
img: 0 idx 10 class  1 equal  True
img: 0 idx 11 class  0 equal  True
img: 0 idx 12 class  1 equal  True
img: 0 idx 13 class  1 equal  True
img: 0 idx 14 class  3 equal  True
img: 0 idx 15 class  0 equal  True
img: 0 idx 16 class  3 equal  True
img: 0 idx 17 class  2 equal  True
img: 0 idx 18 class  2 equal  True
img: 0 idx 19 class  2 equal  True
img: 0 idx 20 class  2 equal  True
img: 0 idx 21 class  0 equal  True
img: 0 idx 22 class  0 equal  True
img: 0 idx 23 class  0 equal  True
img: 0 idx 24 class  0 equal  True
img: 0 idx 25 class  0 equal  True
img: 0 idx 26 class  0 equal  True
img: 0 idx 27 class  0 equal  True
img: 0 idx 28 class  0 equal  

In [21]:
for i in range(32):
    print(gauss_scatt[0,0,i].shape)
    print(gauss_scatt[0,0,i].eval())
          # prob_sum[0,2].eval()

(128, 128)
[[9.8243e-07 1.2426e-06 1.5605e-06 ... 8.6883e-18 4.5394e-18 2.3549e-18]
 [1.0295e-06 1.3020e-06 1.6352e-06 ... 9.1042e-18 4.7566e-18 2.4677e-18]
 [1.0781e-06 1.3635e-06 1.7124e-06 ... 9.5340e-18 4.9812e-18 2.5842e-18]
 ...
 [2.9052e-06 3.6744e-06 4.6145e-06 ... 2.5692e-17 1.3423e-17 6.9638e-18]
 [2.8192e-06 3.5657e-06 4.4781e-06 ... 2.4932e-17 1.3026e-17 6.7579e-18]
 [2.7342e-06 3.4581e-06 4.3430e-06 ... 2.4180e-17 1.2633e-17 6.5540e-18]]
(128, 128)
[[3.2667e-05 4.8889e-05 6.9581e-05 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 [3.4049e-05 5.0958e-05 7.2525e-05 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 [3.5469e-05 5.3083e-05 7.5549e-05 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 ...
 [5.6818e-05 8.5034e-05 1.2102e-04 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 [5.4964e-05 8.2258e-05 1.1707e-04 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 [5.3138e-05 7.9526e-05 1.1318e-04 ... 0.0000e+00 0.0000e+00 0.0000e+00]]
(128, 128)
[[8.7606e-06 9.6187e-06 1.0521e-05 ... 1.8445e-13 1.2613e-13 8.5926e-14]
 [9.31

# Generate 3D Pred Array Tensors

###  Create class score tensor
 1- Class_Score tensor:   batch_size x  num_dectections x num_classes
        
        2- Output_ROIs :         batch_size x  num_detections  x 4    
 3- mrcnn_bboxes:         batch_size x  num_detections  x num_clases x 4
 
###  Generate pred_array: tensor [ seq_id,  max_score,  bbox coordinates, class_id]
  1 -  Get max prediction score and  corresponding class for each ROI
  2 -  Concat sequence id, max_score, bbox coordinates and class id 
  
###  Generate meshgrid of image_id (batch grid) and bbox_ids  (roi_grid)
  Generate scatter indices to scatter bbox information by image_id and class_id 
  The the highest score for each detection (highest score amongst classes) and the class corresponding to the highest score   

###  Split Box/ Class info for each Image dim(0) by class and place into new tensor using tf.scatter_nd

    Output is converted to NUMN_BATCHES x NUM_CLASSES x NUM_ROIS x columns(7)

###  Sort pred_scatter by score 
    Sort each slice based on column  1 (score) using top_k
    Generate calss_grid, batch_grid and roi-grid based using mesh_grid 
    Stack results to generate gather indices 
    Generate final result - pred_tensor - using tf.gather_nd()         

In [3]:
mrcnn_class_init_0 = tf.Variable(tf.random_uniform([num_rois-10, num_classes], minval=0, maxval=1  , dtype=tf.float32, seed=1234), name = 'var')
mrcnn_class_init_1 = tf.Variable(tf.random_uniform([num_rois-4 , num_classes], minval=0, maxval=1  , dtype=tf.float32, seed=1234), name = 'var')
mrcnn_class_init_2 = tf.Variable(tf.random_uniform([num_rois-2 , num_classes], minval=0, maxval=1  , dtype=tf.float32, seed=1234), name = 'var')

output_rois_init_0 = tf.Variable(tf.random_uniform([num_rois-10, 4], minval=0, maxval=128, dtype=tf.float32, seed=1234), name = 'var')
output_rois_init_1 = tf.Variable(tf.random_uniform([num_rois-4 , 4], minval=0, maxval=128, dtype=tf.float32, seed=1234), name = 'var')
output_rois_init_2 = tf.Variable(tf.random_uniform([num_rois-2 , 4], minval=0, maxval=128, dtype=tf.float32, seed=1234), name = 'var')

mrcnn_bboxes       = tf.Variable(tf.random_uniform([batch_size, num_rois, num_classes, 4], minval=0, maxval=128, dtype=tf.float32, seed=1234), name = 'var')

init = tf.global_variables_initializer().run()

mrcnn_class_filler_0 = tf.zeros([10, num_classes], dtype = tf.float32)
mrcnn_class_filler_1 = tf.zeros([4, num_classes], dtype = tf.float32)
mrcnn_class_filler_2 = tf.zeros([2, num_classes], dtype = tf.float32)
mrcnn_class_0 = tf.concat([mrcnn_class_init_0, mrcnn_class_filler_0],axis = 0)
mrcnn_class_1 = tf.concat([mrcnn_class_init_1, mrcnn_class_filler_1],axis = 0)
mrcnn_class_2 = tf.concat([mrcnn_class_init_2, mrcnn_class_filler_2],axis = 0)
print(' mrcnn_class 0 shape: ', mrcnn_class_0.shape)
print(' mrcnn_class 1 shape: ', mrcnn_class_1.shape)
print(' mrcnn_class 2 shape: ', mrcnn_class_2.shape)

mrcnn_class   = tf.stack([mrcnn_class_0, mrcnn_class_1, mrcnn_class_2], axis = 0 )
print(' mrcnn_class shape: ', mrcnn_class.shape)
# print(mrcnn_class.eval())

output_rois_filler_0 = tf.zeros([10, 4], dtype = tf.float32)
output_rois_filler_1 = tf.zeros([4 , 4], dtype = tf.float32)
output_rois_filler_2 = tf.zeros([2 , 4], dtype = tf.float32)
# output_rois_filler = tf.zeros([batch_size, 2, 4], dtype = tf.float32)
output_rois_0        = tf.concat([output_rois_init_0, output_rois_filler_0],axis = 0)
output_rois_1        = tf.concat([output_rois_init_1, output_rois_filler_1],axis = 0)
output_rois_2        = tf.concat([output_rois_init_2, output_rois_filler_2],axis = 0)
print(' output_rois 0 shape: ', output_rois_0.shape)
print(' output_rois 1 shape: ', output_rois_1.shape)
print(' output_rois 2 shape: ', output_rois_2.shape)

output_rois   = tf.stack([output_rois_0, output_rois_1, output_rois_2], axis = 0 )

print(' output_rois shape: ', output_rois.shape)
print(output_rois.eval())


one   = tf.ones([num_rois, 4])
test_bboxes = tf.stack([one, one+1, one+2, one+3], axis = 0) 


pred_classes     = tf.argmax(mrcnn_class,axis=-1, output_type = tf.int32)
pred_classes_exp = tf.to_float(tf.expand_dims(pred_classes,axis=-1))
print('  pred_classes (classes with highest scores):', pred_classes.get_shape())
print(pred_classes.eval())
print('  predclasses expanded shape:', pred_classes_exp.get_shape())

pred_scores  = tf.reduce_max(mrcnn_class,axis=-1, keepdims=True)
print('  pred_scores shape', pred_scores.shape)
# print(pred_scores.eval())

batch_grid, roi_grid = tf.meshgrid( tf.range(batch_size, dtype=tf.int32), tf.range(num_rois, dtype=tf.int32), indexing = 'ij' )

print('roi_grid shape',tf.shape(roi_grid).eval())
print(roi_grid.eval())
print('batch_grid  shape',tf.shape(batch_grid).eval())
print(batch_grid.eval())

bbox_idx        = tf.to_float(tf.expand_dims(roi_grid, axis = -1))

pred_array = tf.concat([ bbox_idx, pred_scores , output_rois, pred_classes_exp], axis=-1)

print('  pred_array shape', pred_array.shape)
print(pred_array[0].eval())
print('------')
print(pred_array[1].eval())
print('------')
print(pred_array[2].eval())

scatter_classes = tf.stack([batch_grid , pred_classes, roi_grid],axis = -1)
print('-- scatter to classes ----')
print('scatter_classes shape', scatter_classes.get_shape())
print(scatter_classes.eval())

pred_scatt = tf.scatter_nd(scatter_classes, pred_array, [batch_size, num_classes, num_rois,7])
print('pred_scatter shape is ', pred_scatt.get_shape())
np.set_printoptions(linewidth=100, precision=4)
print('------------')
print(pred_scatt[0].eval())
print('------------')
np.set_printoptions(linewidth=100, precision=4)
print(pred_scatt[1].eval())
print('------------')
print(pred_scatt[2].eval())

# equality = tf.equal(pred_scatt[0,0,0], pred_scatt2[0,0,0])
# print(tf.reduce_all(equality).eval())

sort_vals, sort_inds = tf.nn.top_k(pred_scatt[:,:,:,1], k=pred_scatt.shape[2])

# print(' Sort vals: ',tf.shape(sort_vals).eval())
# print(sort_vals.eval())
print()
print(' Sort inds: ',tf.shape(sort_inds).eval())
print(sort_inds.eval())

class_grid, batch_grid, dtct_grid = tf.meshgrid(tf.range(num_classes),tf.range(batch_size), tf.range(num_rois))
gather_inds = tf.stack([batch_grid , class_grid, sort_inds],axis = -1)

# print('class_grid', type(class_grid), 'shape',tf.shape(class_grid).eval())
# print(class_grid.eval())

# print('batch_grid', type(batch_grid), 'shape',tf.shape(batch_grid).eval())
# print(batch_grid.eval())

# print('class_grid', type(dtct_grid), 'shape',tf.shape(dtct_grid).eval())
# print(dtct_grid.eval())

print('-- stack results ----')
print('pred_scatt ', type(pred_scatt), 'shape',tf.shape(pred_scatt).eval())
print('gather_inds', type(gather_inds), 'shape',tf.shape(gather_inds).eval())
print(gather_inds.eval())


pred_tensor = tf.gather_nd(pred_scatt, gather_inds)

print()
print('-- gather_nd results (A-boxes sorted by score ----')
print(' srtd_boxes shape : ', tf.shape(pred_tensor).eval())
print(pred_tensor.eval())

 mrcnn_class 0 shape:  (32, 4)
 mrcnn_class 1 shape:  (32, 4)
 mrcnn_class 2 shape:  (32, 4)
 mrcnn_class shape:  (3, 32, 4)
 output_rois 0 shape:  (32, 4)
 output_rois 1 shape:  (32, 4)
 output_rois 2 shape:  (32, 4)
 output_rois shape:  (3, 32, 4)
[[[108.5833  41.4171  39.2576   8.9216]
  [116.9736  21.8213  36.2715  45.6026]
  [ 69.3192   9.6329   9.5347 110.8421]
  [ 71.5596   1.1958   8.5777  63.7113]
  [ 38.2599 103.6086  84.2878  61.6374]
  [ 19.4185  75.6502  42.8045  19.8604]
  [ 31.423   17.8502 126.3289 115.5957]
  [ 19.4829   2.9421 101.6725  85.5338]
  [ 97.5796 122.5435 104.1375  26.6011]
  [110.8402  12.9817  28.9916   4.0611]
  [ 23.823   55.0586  15.4241  31.1665]
  [ 45.7513  24.8754  24.5708  37.1656]
  [106.9807 118.7934 106.3466   7.4645]
  [ 39.4174 116.8137  50.6049 100.7003]
  [ 49.8244  12.3429  81.8621  85.3981]
  [ 99.8521  70.4273  73.7088  58.9241]
  [ 69.2067  70.3032  27.8131  81.6126]
  [ 61.6912   3.5655 122.1108 118.7579]
  [102.1897  90.9454 105.3337 

[[[  0.       0.8483 108.5833  41.4171  39.2576   8.9216   0.    ]
  [  1.       0.9139 116.9736  21.8213  36.2715  45.6026   0.    ]
  [  0.       0.       0.       0.       0.       0.       0.    ]
  [  3.       0.5591  71.5596   1.1958   8.5777  63.7113   0.    ]
  [  0.       0.       0.       0.       0.       0.       0.    ]
  [  0.       0.       0.       0.       0.       0.       0.    ]
  [  0.       0.       0.       0.       0.       0.       0.    ]
  [  0.       0.       0.       0.       0.       0.       0.    ]
  [  0.       0.       0.       0.       0.       0.       0.    ]
  [  9.       0.8659 110.8402  12.9817  28.9916   4.0611   0.    ]
  [  0.       0.       0.       0.       0.       0.       0.    ]
  [ 11.       0.3574  45.7513  24.8754  24.5708  37.1656   0.    ]
  [  0.       0.       0.       0.       0.       0.       0.    ]
  [  0.       0.       0.       0.       0.       0.       0.    ]
  [  0.       0.       0.       0.       0.       0.       0. 

###  Display for visual check

In [4]:
np.set_printoptions(linewidth=100, precision=4)
print('scatter shape is ', pred_scatt.get_shape())
print('pred_tensor shape is ', pred_tensor.get_shape() )
img  = 0
# print(pred_scatt[img,0].eval())
print(pred_tensor[img,0].eval())

print('------------')
# print(pred_scatt[img, 1].eval())
print(pred_tensor[img,1].eval())

print('------------')
# print(pred_scatt[img,2].eval())
print(pred_tensor[img, 2].eval())

print('------------')
# print(pred_scatt[img,3].eval())
print(pred_tensor[img, 3].eval())


scatter shape is  (3, 4, 32, 7)
pred_tensor shape is  (3, 4, 32, 7)
[[  1.       0.9139 116.9736  21.8213  36.2715  45.6026   0.    ]
 [  9.       0.8659 110.8402  12.9817  28.9916   4.0611   0.    ]
 [  0.       0.8483 108.5833  41.4171  39.2576   8.9216   0.    ]
 [ 15.       0.7801  99.8521  70.4273  73.7088  58.9241   0.    ]
 [ 21.       0.6092  77.9712  18.8217  15.8669  12.7541   0.    ]
 [  3.       0.5591  71.5596   1.1958   8.5777  63.7113   0.    ]
 [ 11.       0.3574  45.7513  24.8754  24.5708  37.1656   0.    ]
 [  0.       0.       0.       0.       0.       0.       0.    ]
 [  0.       0.       0.       0.       0.       0.       0.    ]
 [  0.       0.       0.       0.       0.       0.       0.    ]
 [  0.       0.       0.       0.       0.       0.       0.    ]
 [  0.       0.       0.       0.       0.       0.       0.    ]
 [  0.       0.       0.       0.       0.       0.       0.    ]
 [  0.       0.       0.       0.       0.       0.       0.    ]
 [  0.  

###  Build indicies to gather bounding boxes from bboxes_4d corrsponding to predicted class
#### Only used if we want to use mrcnn_bboxes (batch_size, num_rois, num_classes, 4)

batch_size x nuum_detections x 4

In [1]:
###  Build indicies to gather bounding boxes from bboxes_4d corrsponding to predicted class
###  Only used if we want to use mrcnn_bboxes (batch_size, num_rois, num_classes, 4)

# gather_boxes = tf.stack([batch_grid, roi_grid, pred_classes, ], axis = -1)

# print('-- gather_boxes  ----')
# print('gather_boxes inds', type(gather_boxes), 'shape',tf.shape(gather_boxes).eval())
# print(gather_boxes.eval())

# mrcnn_bboxes_selected = tf.gather_nd(mrcnn_bboxes, gather_boxes)
# print(' padding required for output_rois : ', mrcnn_bboxes_selected.get_shape())
# print(mrcnn_bboxes_selected[0].eval())

# print(' output_rois shape ', output_rois.get_shape())
# print(pred_classes[0].eval())
# print(output_rois[0].eval())



In [ ]:
# tf.count_nonzero(pred_tensor[0,0]).eval()

In [ ]:
a00 = pred_tensor
print(a00[:,:,:,0].get_shape())

a00_cnt = tf.count_nonzero(a00[:,:,:,0],axis = -1)
print(a00_cnt.get_shape())
a00_cnt.eval()

###  Sort each slice based on class id (obsolete)

#### Now we split based on class id first then sort based on score - no need to sort on class beforehand

In [ ]:
# sort_vals, sort_inds = tf.nn.top_k(-pred_array[:,:,6], k=pred_array.shape[1])

# print(tf.shape(sort_vals).eval())
# print(sort_vals.eval())
# print()
# print(tf.shape(sort_inds).eval())
# print(sort_inds.eval())

###  Scatter to class layers - Loop / Dynamic Partition method  (Obsolete)
 
We are scattering using the scatter_nd_update method

In [ ]:
print('shape of srtd_boxes ', srtd_boxes.shape)
partition_inds = tf.to_int32(srtd_boxes[...,6])
print(partition_inds.eval())


batch_stack = []
for i in range(batch_size):
    
    splitted = tf.dynamic_partition(srtd_boxes[i], partition_inds[i], num_classes)

    for idx, split in enumerate(splitted):
#         print(' item ', idx,' shape ',split.shape, 'shape:', tf.shape(split).eval(), 'rank: ', tf.rank(split).eval())
        padding = num_rois - tf.shape(split)[0]
        split = tf.pad(split,[[0,padding],[0,0]]) 
#         print(' num of padding rows tt add ', padding.eval())        
#         print(' shape after padding : ', tf.shape(split).eval())
#         print(split.eval())
        splitted[idx] = split
    stacked = tf.stack(splitted, axis = 0)
    batch_stack.append(stacked)

batch_tensor = tf.stack(batch_stack, axis = 0)
print(' batch _tensor shape is ', tf.shape(batch_tensor).eval())
print(batch_tensor[0].eval())    


## Experimental Code

###  Experiment for sort ordering 

In [ ]:
indices = tf.constant([
            [[0,2],[0,1],[0,0]],
            [[1,2],[1,1],[1,0]],
            [[2,0],[2,1],[2,1]],
          ])
params = tf.constant( [
          [['0-00', '0-01', '0-02', '0-03'], 
           ['0-10', '0-11', '0-12', '0-13'],
           ['0-20', '0-21', '0-22', '0-23'],
           ['0-30', '0-31', '0-32', '0-33']],
    
          [['1-00', '1-01', '1-02', '1-03'], 
           ['1-10', '1-11', '1-12', '1-13'],
           ['1-20', '1-21', '1-22', '1-23'],
           ['1-30', '1-31', '1-32', '1-33']],
    
          [['2-00', '2-01', '2-02', '2-03'], 
           ['2-10', '2-11', '2-12', '2-13'],
           ['2-20', '2-21', '2-22', '2-23'],
           ['2-30', '2-31', '2-32', '2-33']]    
         ])
print(params.shape, '   ', indices.shape)
res = tf.gather_nd(params, indices)
res.eval()



###  Experiment with GATHER() for splitting into classes

In [ ]:
# indices = tf.constant([
#             [0,2,2],[0,1,1],[0,0,0],
#             [1,2,2],[1,1,1],[1,0,0],
#             [2,0,0],[2,1,1],[2,1,1],
#            ])
# indices = tf.constant([ [0,2],[0,1],[0,0],[1,2],[1,1],[1,0],[1,2],[1,3], [2,3]] ) # 9 x 2
indices = tf.constant([
           [
            [[0,2],[0,1],[0,0],[1,2],[2,3]],
            [[1,1],[1,0],[1,2],[1,3],[1,1]]
           ]
           ])
params = tf.constant( [
          [['0-00', '0-01', '0-02', '0-03', '0-04', '0-05', '0-06'], 
           ['0-10', '0-11', '0-12', '0-13', '0-14', '0-15', '0-16'],
           ['0-20', '0-21', '0-22', '0-23', '0-24', '0-25', '0-26'],
           ['0-30', '0-31', '0-32', '0-33', '0-34', '0-35', '0-36'],
           ['0-40', '0-41', '0-42', '0-43', '0-44', '0-45', '0-46']],
                                                          
          [['1-00', '1-01', '1-02', '1-03', '1-04', '1-05', '1-06'], 
           ['1-10', '1-11', '1-12', '1-13', '1-14', '1-15', '1-16'],
           ['1-20', '1-21', '1-22', '1-23', '1-24', '1-25', '1-26'],
           ['1-30', '1-31', '1-32', '1-33', '1-34', '1-35', '1-36'],
           ['1-40', '1-41', '1-42', '1-43', '1-44', '1-45', '1-46']],
                                                          
          [['2-00', '2-01', '2-02', '2-03', '2-04', '2-05', '2-06'], 
           ['2-10', '2-11', '2-12', '2-13', '2-14', '2-15', '2-16'],
           ['2-20', '2-21', '2-22', '2-23', '2-24', '2-25', '2-26'],
           ['2-30', '2-31', '2-32', '2-33', '2-34', '2-35', '2-36'],
           ['2-40', '2-41', '2-42', '2-43', '2-44', '2-45', '2-46']]    
         ])
print(' params sahape: ', params.shape, ' parms[:axis]', params.shape[:0], params.shape[:1], params.shape[:2], params.shape[:3])
print(' params sahape: ', params.shape, ' parms[:axis]', params.shape[0:], params.shape[1:], params.shape[2:], params.shape[3:])
print(' indices.shape  ', indices.shape)
res = tf.gather_nd(params, indices)
print('result shape   ', res.get_shape())
res.eval()

###  Experiment with SCATTER() for splitting into classes

In [ ]:
# del indices, updates, shape

indices =  tf.constant([
                         #|
                         #| 
                        [[0,0,0], [0,3,0], [0,2,0], [0,1,0],[0,1,1],[0,1,2],[0,2,1],[0,3,2] ],
                        [[1,0,0], [1,3,0], [1,2,0], [1,1,0],[1,1,1],[1,1,2],[1,2,1],[1,3,2] ],
                        [[2,0,0], [2,3,0], [2,2,0], [2,1,0],[2,1,1],[2,1,2],[2,2,1],[2,3,2] ]
])
                            #<--- each one corrsponds to an entry in  the updates array [row , position in resulting array]
    
#                        [[1,1], [3,1], [2,1], [1,1],[1,1],[1,2],[2,1],[3,2] ],
#                        [[0,0], [3,0], [2,0], [1,0],[1,1],[1,2],[2,1],[3,2] ]
    
###----------------------------------------------------
### Our bounding box array
###---------------------------------------------------
updates = tf.constant( [
          [[1000, 1001, 1002, 1003, 1004, 1005, 1006], 
           [1010, 1011, 1012, 1013, 1014, 1015, 1016],
           [1020, 1021, 1022, 1023, 1024, 1025, 1026],
           [1030, 1031, 1032, 1033, 1034, 1035, 1036],
           [1040, 1041, 1042, 1043, 1044, 1045, 1046],
           [1050, 1051, 1052, 1053, 1054, 1055, 1056],
           [1060, 1061, 1062, 1063, 1064, 1065, 1066],
           [1070, 1071, 1072, 1073, 1074, 1075, 1076] ],
          
          [[2000, 2001, 2002, 2003, 2004, 2005, 2006], 
           [2010, 2011, 2012, 2013, 2014, 2015, 2016],
           [2020, 2021, 2022, 2023, 2024, 2025, 2026],
           [2030, 2031, 2032, 2033, 2034, 2035, 2036],
           [2040, 2041, 2042, 2043, 2044, 2045, 2046],
           [2050, 2051, 2052, 2053, 2054, 2055, 2056],
           [2060, 2061, 2062, 2063, 2064, 2065, 2066],
           [2070, 2071, 2072, 2073, 2074, 2075, 2076]] , 

          [[3000, 3001, 3002, 3003, 3004, 3005, 3006], 
           [3010, 3011, 3012, 3013, 3014, 3015, 3016],
           [3020, 3021, 3022, 3023, 3024, 3025, 3026],
           [3030, 3031, 3032, 3033, 3034, 3035, 3036],
           [3040, 3041, 3042, 3043, 3044, 3045, 3046],
           [3050, 3051, 3052, 3053, 3054, 3055, 3056],
           [3060, 3061, 3062, 3063, 3064, 3065, 3066],
           [3070, 3071, 3072, 3073, 3074, 3075, 3076] ]    
         ])                       

zeros = tf.zeros([3,4,8,7], dtype=tf.int32)
ref   = tf.Variable(zeros)
scatt1= tf.Variable(zeros)
shape = tf.constant([3,4, 8, 7])
 
# init = tf.global_variables_initializer().run()
# print('indices shape ', indices.shape,  'indices_shape [-1]', indices.shape[-1],  'indices_shape [:-1]', indices.shape[:-1])
# print('updates.shape ', updates.shape)
# print('ref     shape ', ref.get_shape(), ref)
# print('scatt1  shape ', scatt1.get_shape(), scatt)

scatt1 = tf.scatter_nd(indices, updates, shape)
# print('scatt1  shape ', scatt1.get_shape(), scatt)
# print(scatt1[0].eval())
# print('------------')
# print(scatt1[1].eval())
# print('------------')
# print(scatt1[2].eval())

# scatt = tf.scatter_nd_update(ref, indices, updates)
# print('scatter shape is ', scatt.get_shape(), scatt)
# print(scatt.eval())

print()


In [ ]:
indices = tf.constant([
                        [[0,0], [3,0], [2,0], [1,0]]
#                     ,[[1,3], [1,3], [1,2], [1,0]]
                      ])
updates = tf.constant([
                       [[5, 5, 5, 5, 0], 
                        [6, 6, 6, 6, 3],
                        [7, 7, 7, 7, 2], 
                        [8, 8, 8, 8, 1]]
                       
#                      , [[5, 5, 5, 3], 
#                         [6, 6, 6, 3],
#                         [7, 7, 7, 2], 
#                         [8, 8, 8, 0]]
                       ])
shape = tf.constant([4, 4, 5])
print(indices.shape, 'inputs shape', updates.shape, shape.shape)
scatter = tf.scatter_nd(indices, updates, shape)
print('scatter shape is ', scatter.get_shape())
print(scatter.eval())

In [ ]:
num_detections = 8
# batch_size is  3
gt_class_ids = tf.constant([[1,2,3,0,0,0,0,0],[3,3,3,0,0,0,0,0],[1,2,2,0,0,2,0,0]])
gt_bboxes    = tf.random_uniform([batch_size,3,4], maxval = 127, dtype=tf.int32)


gt_classes_exp = tf.to_float(tf.expand_dims(gt_class_ids ,axis=-1))
print('    gt_classes_exp: ' ,gt_classes_exp.get_shape())
print(gt_classes_exp.eval())


zeros        = tf.zeros([batch_size,5,4], dtype = tf.int32)
gt_bboxes    = tf.concat([gt_bboxes, zeros], axis = 1)
gt_bboxes    = tf.to_float(gt_bboxes)
print('\n    gt_bboxes: ' ,gt_bboxes.get_shape())
print(gt_bboxes.eval())

mask = tf.greater(gt_class_ids,0)
print(mask.eval())

gt_scores     = tf.where(mask, tf.ones_like(gt_class_ids), tf.zeros_like(gt_class_ids))
gt_scores_exp = tf.to_float(tf.expand_dims(gt_scores, axis=-1))
print('\n    gt_scores ', gt_scores_exp.get_shape())
print(gt_scores_exp.eval())

batch_grid, bbox_grid = tf.meshgrid( tf.range(batch_size    , dtype=tf.int32), 
                                    tf.range(num_detections , dtype=tf.int32), indexing = 'ij' )

print('\n    bbox_grid   ', type(bbox_grid)  , 'shape', bbox_grid.get_shape())
print(bbox_grid.eval())
print('\n    batch_grid ', type(batch_grid), 'shape', batch_grid.get_shape())
print(batch_grid.eval())
 
bbox_idx_zeros  = tf.zeros_like(bbox_grid)
bbox_idx        = tf.where(mask, bbox_grid , bbox_idx_zeros)
bbox_idx        = tf.to_float(tf.expand_dims(bbox_idx, axis = -1))    
print('    bbox_idx', type(bbox_idx), 'shape', bbox_idx.get_shape())
print(bbox_idx.eval())

In [ ]:
gt_array        = tf.concat([bbox_idx, gt_scores_exp , gt_bboxes, gt_classes_exp], axis=2)
print('    gt_array  ',  type(gt_array), gt_array.shape)
print(gt_array.eval())


In [ ]:
scatter_classes = tf.stack([batch_grid , gt_class_ids, bbox_grid],axis = -1)
print('\n    -- stack results ----')
print('\n    scatter_classes', type(scatter_classes), 'shape',tf.shape(scatter_classes).eval())
print(scatter_classes.eval())

In [ ]:
gt_scatter = tf.scatter_nd(scatter_classes, gt_array, [batch_size, num_classes, num_detections,7])
print('    gt_tensor shape is ', gt_scatter.get_shape(), gt_scatter)
gt_scatter.eval()

In [ ]:
sort_vals, sort_inds = tf.nn.top_k(gt_scatter[:,:,:,0], k=gt_scatter.shape[2])
print('    sort vals shape : ', sort_vals.get_shape())
print(sort_vals.eval())
print('    sort inds shape : ', sort_inds.get_shape())
print(sort_inds.eval())

In [ ]:
# build gathering indexes to use in sorting 
class_grid, batch_grid, bbox_grid = tf.meshgrid(tf.range(num_classes),tf.range(batch_size), tf.range(num_detections))

print('    class_grid  ', type(class_grid) , 'shape', class_grid.get_shape())
print(class_grid.eval())
print('    batch_grid  ', type(batch_grid) , 'shape', batch_grid.get_shape())
print(class_grid.eval())
print('    bbox_grid    ', type(bbox_grid) , 'shape', bbox_grid.get_shape())
print(bbox_grid.eval())

In [ ]:
gather_inds = tf.stack([batch_grid , class_grid, sort_inds],axis = -1)
print('    -- pred_tensor results (A-boxes sorted by score ----')
print('    gatehr_inds ', gather_inds.get_shape())
print(gather_inds.eval())

gt_tensor = tf.gather_nd(gt_scatter, gather_inds)
print('    -- pred_tensor results (A-boxes sorted by score ----')
print('    pred_tensor ', gt_tensor.get_shape())
print(gt_tensor.eval())

###  Split Box/ Class info for each Image dim(0) by class and place into new tensor using tf.scatter_nd_update
###  obsolete -- use tf.scatter_nd instead
    Output is converted to NUMN_BATCHES x NUM_CLASSES x num_rois x columns 

In [ ]:
# scatt1 = tf.scatter_nd(scatter_inds, pred_array, [batch_size, num_classes, num_rois])
# print('scatt1  shape ', scatt1.get_shape(), scatt)

# zeros = tf.zeros([batch_size,num_classes, num_rois,7], dtype=tf.float32)
# ref   = tf.Variable(zeros)
# init = tf.global_variables_initializer().run()

# pred_scatt_old = tf.scatter_nd_update(ref, scatter_classes, pred_array)


# print('scatter shape is ', pred_scatt.get_shape(), pred_scatt)
# print(pred_scatt[0].eval())
# print('------------')
# print(pred_scatt[1].eval())
# print('------------')
# print(pred_scatt[2].eval())



###  Stack non-zero bbox infromaton from pred_array into stacked_array --- method 2 (obsolete)

In [ ]:
# sep = tf.unstack(pred_tensor)
# print(len(sep))
# all_boxes = []
# for sub_sep in sep:
#     p_reduced = tf.reduce_sum(tf.abs(sub_sep), axis=-1)
#     print(' p_redcued shape ',p_reduced.shape)
# #     print(p_reduced.eval())
#     non_zeros = tf.cast(p_reduced, tf.bool)
#     print(' mask shape' ,non_zeros.shape) 
#     # print(non_zeros.eval())
#     boxes = tf.boolean_mask(sub_sep, non_zeros,axis = 0)
#     print(' boxes shape ',boxes.get_shape())
#     print(boxes.eval())

#     all_boxes.append(boxes)
# print(len(all_boxes))
# stacked_tensor = tf.stack(all_boxes)
# print('stacked shape',stacked_tensor.get_shape())

# p_reduced = tf.reduce_sum(tf.abs(pred_tensor), axis=-1)
# print(' p_redcued shape ',p_reduced.shape)
# print(p_reduced.eval())
# non_zeros = tf.cast(p_reduced, tf.bool)
# print(' mask shape' ,non_zeros.shape) 
# print(non_zeros.eval())
# boxes = tf.boolean_mask(pred_tensor, non_zeros,axis = 0)
# print(boxes.shape)
# print(boxes.eval())
# print(keepers[1].eval())

#### Another (slower) method for creating a meshgrid with the following shape 
This method is slightly slower  - build meshgrid in the following dimensions to allow for tensorflow Multivariate Normal Dist

    image width x image height  x batch_size x num_rois_per_image x 2    (8 x 16 x 3 x 32 x 2)

In [15]:
# rois_per_image  = 32
# img_h  = 8
# img_w  = 16
# X = tf.range(img_w, dtype=tf.int32)
# Y = tf.range(img_h, dtype=tf.int32)
# X, Y = tf.meshgrid(X, Y)
# print( X.get_shape(), Y.get_shape())
# # print( ' X : \n',X.eval())
# # print( ' Y : \n',Y.eval())

# twos_X = tf.tile(tf.expand_dims(X, axis = -1),[1,1,3])
# twos_Y = tf.tile(tf.expand_dims(Y, axis = -1),[1,1,3])
# # print(' twos.shape X',twos_X.shape)
# # print(' twos_shape Y',twos_Y.shape)

# thre_X = tf.tile(tf.expand_dims(twos_X, axis = -1),[1,1,1,32])
# thre_Y = tf.tile(tf.expand_dims(twos_Y, axis = -1),[1,1,1,32])
# # print(' twos_X shape ', twos_X.shape, ' thre_X shape ',thre_X.shape)
# # print(' twos_Y shape ', twos_Y.shape, ' thre_Y shape ',thre_Y.shape)

# # stack the X and Y grids 
# pos_grid_2 = tf.to_float(tf.stack([thre_X,thre_Y], axis = -1))
# print(pos_grid_2.get_shape())